# Physical risk economic impact model

This notebook computes the effect of the propagation of physical shocks along value chains using an input-output model with the possibility of substitution of input sources among different geographies

In [1]:
import pandas as pd
import numpy as np

First, upload the data and manage the main entities (sectors, geographies)

In [2]:
# Set the input file path
input_file = "MPSAM_2905.xlsx"

# Read sectors and geographies data
sectors = pd.read_excel(input_file, sheet_name=0, usecols="B", skiprows=2, header=None, nrows=1177)
sectors.columns = ["Sector"]  # Explicitly set column name

geographies = pd.read_excel(input_file, sheet_name=0, usecols="A", skiprows=2, header=None, nrows=1177)
geographies.columns = ["Geography"]  # Explicitly set column name

# Number of unique sectors and geographies
sec = sectors["Sector"].nunique()
m = geographies["Geography"].nunique()

# Total number of entries
n = sec * m

# Read matrices Z, VA, and FD from the Excel file
Z = pd.read_excel(input_file, sheet_name=0, usecols="C:ASI", skiprows=2,  nrows=1177, header=None).values
VA = pd.read_excel(input_file, sheet_name=0, usecols="C:ASI", skiprows=1179, nrows=4, header=None).values
FD = pd.read_excel(input_file, sheet_name=0, usecols="ASM:ASP", skiprows=2,  nrows=1177, header=None).values

For the moment shocks are read from excel file. **These should become parameteres decided by the user**

In [3]:
# Read demand (d) and supply (p) shock data
d = pd.read_excel(input_file, sheet_name=0, usecols="ASR", skiprows=2, nrows=1177, header=None).values
p = pd.read_excel(input_file, sheet_name=0, usecols="ASS", skiprows=2, nrows=1177, header=None).values


Compute the macro-dimensions and the direct effects on production capacity and final demand

In [4]:
# Compute value added and final demand
VA = VA.sum(axis=0)
FD = FD.sum(axis=1)

# Compute total output X
X = Z.sum(axis=0) + VA

# Compute the technical coefficient matrix A
A = (Z/ X)

# Compute the Leontief inverse L
L = np.linalg.inv(np.eye(n) - A)

# Aggregate technical coefficients by sectors
AG = np.zeros((sec, n))

# Use the correct column name "Sector" instead of an index
unique_sectors = sectors["Sector"].unique()
for i in range(sec):
    AG[i, :] = A[sectors["Sector"] == unique_sectors[i], :].sum(axis=0)

# Calculate maximum final demand and maximum output after shocks
fd_max = FD * (1 - d.flatten())
X_max = X * (1 - p.flatten())

# Compute initial disrupted output
X_md = L @ fd_max

Reallocation iterative algorithm: given the supply and demand shocks, the economy adjusts iteratively until the best suboptimal resources allocation is reached

In [5]:

# Set parameters for the simulation
lambda_param = 1  # reallocation capacity parameter: 0 = no reallocation capacity , 1 = full reallocation capacity
tol = 0.01        # convergence tolerance

# Initialize variables for the iteration process
check = False
count = 0

# Iterative adjustment process
while not check:
    # Compute the ratio r for constraints
    r = np.minimum(X_max / X_md, 1)

    # Flatten the p array
    p_flat = p.flatten()

    # Calculate constraints for each sector
    const = np.array([min(np.concatenate([r[Z[:, i] > 0], [1 - p_flat[i]]])) for i in range(n)])

    # Apply constraints to the intermediate demand matrix Z
    Z_const = const * Z
        
    # Calculate the needed intermediate demand
    Z_need = A @ np.diag(X_md)
        
    # Compute excess demand
    exD = Z_need - Z_const

    # Aggregate excess demand by sectors
    exDsecj = np.zeros((sec, n))
    for i in range(sec):
        exDsecj[i, :] = exD[sectors["Sector"] == unique_sectors[i], :].sum(axis=0)
    exDsec = exDsecj.sum(axis=1)

    # Compute the available inventory
    inv = np.maximum(X_max - fd_max - Z_const.sum(axis=1), 0)
    invsec = np.array([inv[sectors["Sector"] == unique_sectors[i]].sum() for i in range(sec)])
        
    # Compute the substitution rate
    sub = invsec / exDsec

    # Initialize reallocation matrix
    reallocation = np.zeros((n, n))
        
    # Perform reallocation based on constraints and excess demand
    for i in range(n):
        sector_idx = np.where(unique_sectors == sectors.iloc[i, 0])[0][0]
        reallocation[i, :] = lambda_param * inv[i] / invsec[sector_idx] * sub[sector_idx] * exDsecj[sector_idx, :]

    # Update the intermediate demand matrix Z
    Z_new = Z_const + reallocation

    # Aggregate the new intermediate demand by sectors
    ZGnew = np.zeros((sec, n))
    for i in range(sec):
        ZGnew[i, :] = Z_new[sectors["Sector"] == unique_sectors[i], :].sum(axis=0)

    # Calculate the new output matrix X_new_m and X_new
    X_new_m = ZGnew / AG
    X_new = np.min(X_new_m, axis=0)

    # Update final demand based on the new output
    fd_new = np.maximum(X_new - Z_new.sum(axis=1), 0)

    # Update the technical coefficient matrix A_new and Leontief inverse L_new
    A_new = (Z_new / X_new)
    L_new = np.linalg.inv(np.eye(n) - A_new)

    # Compute the target output with the new final demand
    X_new_target = L_new @ fd_new

    # Check convergence criteria
    check = np.all(np.abs(X_new - X_new_target) < tol)
    # Update variables for the next iteration
    X_max = X_new
    X_md = X_new_target
    Z = Z_new
    A = A_new
    fd_max = fd_new
    
    # Increment the iteration count
    count += 1

Once the algorithm has converged, compute final outcomes and main impact ratios

In [6]:
# Compute the new value added
VA_new = X_new - Z_new.sum(axis=0)

# Calculate aggregate results
FD_shock = fd_max.sum() - FD.sum()
Prod_shock = -p.flatten() @ X
VA_impact = VA_new.sum() - VA.sum()
Prod_impact = X_new.sum() - X.sum()

# Output the key ratios
VA_impact_ratio = VA_impact / FD_shock
Prod_impact_ratio = Prod_impact / Prod_shock

print(f"Value Added Impact Ratio: {VA_impact_ratio}")
print(f"Production Impact Ratio: {Prod_impact_ratio}")

Value Added Impact Ratio: 1.000000000000067
Production Impact Ratio: 0.7851758251371754
